In [85]:
import os
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# all directories
TRAIN_DIR = 'extracted/cats_and_dogs_filtered/train'
VAL_DIR = 'extracted/cats_and_dogs_filtered/validation'
TRAIN_DIR_CATS = 'extracted/cats_and_dogs_filtered/train/cats'
TRAIN_DIR_DOGS = 'extracted/cats_and_dogs_filtered/train/dogs'
VAL_DIR_CATS = 'extracted/cats_and_dogs_filtered/validation/cats'
VAL_DIR_DOGS = 'extracted/cats_and_dogs_filtered/validation/dogs'

# getting images squared
imageSize = 50

# alpha
lr = 0.001

# momentum
beta_1 = 0.9

# mini-batch size
BATCH_SIZE = 32

# number of epochs
epochs = 10

# total training data and val data
totalTrain = len(os.listdir(TRAIN_DIR_CATS))
totalTrain += len(os.listdir(TRAIN_DIR_DOGS))

totalVal = len(os.listdir(VAL_DIR_CATS))
totalVal += len(os.listdir(VAL_DIR_DOGS))

In [91]:
# for data augmentation since we have a small training set
trainImageGenerator = ImageDataGenerator(
                                        rescale = 1./255.,  # normalization
                                        rotation_range = 40,  # range of rotation
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True,  # fliiping images horizontally
                                        fill_mode = 'nearest'  # any pixel gaps will be filled with the 'nearest' pizel
                                        )

validationImageGenerator = ImageDataGenerator(
                                        rescale = 1./255.
                                        )

In [92]:
# declaring training data
trainingData = trainImageGenerator.flow_from_directory(
                                        batch_size = BATCH_SIZE,
                                        directory = TRAIN_DIR,
                                        shuffle = True,
                                        target_size = (imageSize, imageSize),
                                        class_mode = 'binary'   # only cats or dogs, binary classification
                                        )

validationData = validationImageGenerator.flow_from_directory(
                                        batch_size = BATCH_SIZE,
                                        directory = VAL_DIR,
                                        shuffle = False,
                                        target_size = (imageSize, imageSize),
                                        class_mode = 'binary'
                                        )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [93]:
# printing classes
print(trainingData.class_indices)

{'cats': 0, 'dogs': 1}


In [94]:
# clearing previous models
K.backend.clear_session()

# defining the model
model = K.models.Sequential([
    Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = (imageSize, imageSize, 3)), # 3 for RGB
    MaxPooling2D(pool_size = (2, 2)),
    
    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2)),
    BatchNormalization(),
    
    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2)),
    BatchNormalization(),
    
    Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2)),
    BatchNormalization(),
    
    Dropout(rate = 0.5),        # regularization to reduce variance
    
    Flatten(),
    Dense(units = 256, activation = 'relu'),
    
    Dense(units = 2, activation = 'softmax')
    
], name = 'cats-vs-dogs')

In [95]:
# compiling the model
model.compile (
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
    )

In [96]:
# summary of the model
model.summary()

Model: "cats-vs-dogs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 11, 11, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)        

In [97]:
# fit is overloaded with fit_generator in TensorFlow 2.2+, hence, using fit
model.fit(
    trainingData,
    steps_per_epoch = int(np.ceil(totalTrain / float(BATCH_SIZE))),  # steps per epoch is defined as total by batch size
    epochs = epochs,
    validation_data = validationData,
    validation_steps = int(np.ceil(totalTrain / float(BATCH_SIZE)))
    )

Epoch 1/10
63/63 [==============================] - 16s 248ms/step - loss: 0.7922 - accuracy: 0.5505 - val_loss: 0.6989 - val_accuracy: 0.4900
Epoch 2/10
63/63 [==============================] - 16s 256ms/step - loss: 0.7460 - accuracy: 0.5675 - val_loss: 0.7118 - val_accuracy: 0.5090
Epoch 3/10
63/63 [==============================] - 18s 279ms/step - loss: 0.7260 - accuracy: 0.5780 - val_loss: 0.7648 - val_accuracy: 0.5000
Epoch 4/10
63/63 [==============================] - 15s 241ms/step - loss: 0.7154 - accuracy: 0.5750 - val_loss: 0.6773 - val_accuracy: 0.5760
Epoch 5/10
63/63 [==============================] - 15s 236ms/step - loss: 0.7024 - accuracy: 0.5860 - val_loss: 0.8246 - val_accuracy: 0.5050
Epoch 6/10
63/63 [==============================] - 15s 234ms/step - loss: 0.6804 - accuracy: 0.5930 - val_loss: 0.8059 - val_accuracy: 0.5070
Epoch 7/10
63/63 [==============================] - 15s 237ms/step - loss: 0.6653 - accuracy: 0.6110 - val_loss: 0.6853 - val_accuracy: 0.5680

In [105]:
model.evaluate(validationData, batch_size = BATCH_SIZE)

32/32 [==============================] - 3s 97ms/step - loss: 0.6429 - accuracy: 0.6180


[0.6428672671318054, 0.6179999709129333]

In [104]:
# saving json format of model
modelJSON = model.to_json()
with open('cats_vs_dogs.json', 'w') as file:
    file.write(modelJSON)
    
# saving weights of model
model.save_weights("cats_vs_dogs_weights.h5")

#printing confirmation
print("Saved model to disk")

Saved model to disk


In [106]:
# loading model
jsonFile = open('cats_vs_dogs.json', 'r')
loadedJSONModel = jsonFile.read()
jsonFile.close()
loadedModel = K.models.model_from_json(loadedJSONModel)

# loading weights using this model
loadedModel.load_weights('cats_vs_dogs_weights.h5')

In [108]:
# compiling the loaded model
loadedModel.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

# evaluating the loaded model
loadedModel.evaluate(validationData, batch_size = BATCH_SIZE)

32/32 [==============================] - 3s 95ms/step - loss: 0.6429 - accuracy: 0.6180


[0.6428672671318054, 0.6179999709129333]